### Intersection of mrcanavar, PacBio CCS, and ONT high coverage regions to identify potential CNV

Generating excessive coverage bed files using calculations from mosdepth for both PacBio CCS 15kb_20kb merged and ONT bam files.

HG005vGRCh38_wm_ONT.sort.bam is from /team-variants/wm2.01alignments/HG005vGRCh38_wm_ONT.sort.bam

HG005.GRCh38.haplotagged.bam is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/ChineseTrio/analysis/PacBio_CCS_15kb_20kb_chemistry2_12072020/HG005/HG005.GRCh38.haplotagged.bam


HG005.hg38.300x.bam.bilkentuniv.072319.dups.bed is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/ChineseTrio/analysis/BilkentUni_mrCaNaVaR_GRCh38_07242019/Chinesetrio-HG005.hg38.300x.bam.bilkentuniv.072319.dups.bed.gz

HG005.GRCh38.pbsv.vcf.gz is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/ChineseTrio/analysis/PacBio_CCS_15kb_20kb_chemistry2_12072020/HG005/HG005.GRCh38.pbsv.vcf.gz

convert_mosdepth_to_excessive_coverage.py is at the end of this notebook

In [ ]:
### mosdepth commands

In [ ]:
mosdepth -b 1000 -x --mapq 20 --no-per-base HG005_GRCh38_PacBio_HiFi_1000_window_size_MAPQ20 HG005.GRCh38.haplotagged.bam


samtools view -1 -F 0x100 HG005vGRCh38_wm_ONT.sort.bam -h > HG005vGRCh38_wm_ONT.sort.filtered.bam

mosdepth -b 1000 -x --no-per-base HG005_GRCh38_ONT_filtered_window_size HG005vGRCh38_wm_ONT.sort.filtered.bam

In [ ]:
### Find coverage levels of excessive coverage on CCS and ONT data in R

In [ ]:
chr_1_22 <- c("chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22") 

mosdepth_CCS_15kb_20kb_merged_1000_window_size_GRCh38 = read.delim("HG005_GRCh38_PacBio_HiFi_1000_window_size_MAPQ20.regions.bed", col.names = c("CHR","START","END","DEPTH")) 

mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22_GRCh38 <- mosdepth_CCS_15kb_20kb_merged_1000_window_size_GRCh38[which(mosdepth_CCS_15kb_20kb_merged_1000_window_size_GRCh38[,"CHR"] %in% chr_1_22),]

quantile(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22_GRCh38[,"DEPTH"])

# 0%      25%      50%      75%     100% 
#0.00    42.27    47.89    53.18 15006.65 

IQR(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22_GRCh38[,"DEPTH"])
# 12.35

(quantile(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22_GRCh38[,"DEPTH"])[3]/2)*2.5
# 59.6875

mosdepth_ONT_1000_window_size_GRCh38 = read.delim("HG005_GRCh38_ONT_filtered_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

mosdepth_ONT_1000_window_size_chr_1_22_GRCh38 <- mosdepth_ONT_1000_window_size_GRCh38[which(mosdepth_ONT_1000_window_size_GRCh38[,"CHR"] %in% chr_1_22),]

quantile(mosdepth_ONT_1000_window_size_chr_1_22_GRCh38[,"DEPTH"])

# 0%      25%      50%      75%     100% 
#0.00    51.45    57.59    63.60 42085.22

IQR(mosdepth_ONT_1000_window_size_chr_1_22_GRCh38[,"DEPTH"])
# 12.15

(quantile(mosdepth_ONT_1000_window_size_chr_1_22_GRCh38[,"DEPTH"])[3]/2)*2.5
# 71.9875 

In [ ]:
### HG005_mrcanavar_intersect_ccs_window_size_cnv_threshold_intersect_ont_window_size_cnv_threshold.bed

### What this does: find potential CNVs in HG5 from intersecting coverage files from PacBio HiFi, ONT, and Illumina data. This generates PacBio HiFi excessive coverage bed, intersect with mrCaNaVar dups bed, generates ONT excessive coverage bed, intersects to these all to generate exclusion bed

In [ ]:
python convert_mosdepth_to_excessive_coverage.py --input HG005_GRCh38_PacBio_HiFi_haplotagged_MAPQ20.regions.bed --output HG005_GRCh38_PacBio_HiFi_haplotagged_MAPQ20.regions_excessive_coverage_cnv_threshold.bed --threshold 59.6875

bedtools intersect -a Chinesetrio-HG005.hg38.300x.bam.bilkentuniv.072319.dups.bed -b HG005_GRCh38_PacBio_HiFi_haplotagged_MAPQ20.regions_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_HG005_GRCh38_PacBio_HiFi_1000_window_size_GRCh38_cnv_threshold.bed

python convert_mosdepth_to_excessive_coverage.py --input HG005_GRCh38_ONT_filtered_window_size.regions.bed --output HG005_GRCh38_ONT_filtered_window_size.regions_excessive_coverage_cnv_threshold.bed --threshold 71.9875

bedtools intersect -a mrcanavar_intersect_HG005_GRCh38_PacBio_HiFi_1000_window_size_GRCh38_cnv_threshold.bed -b HG005_GRCh38_ONT_filtered_window_size.regions_excessive_coverage_cnv_threshold.bed > GRCh38_mrcanavar_intersect_HG005_GRCh38_PacBio_HiFi_1000_window_size_GRCh38_cnv_threshold_intersect_ONT_1000_window_size_cnv_threshold.bed

cat GRCh38_mrcanavar_intersect_HG005_GRCh38_PacBio_HiFi_1000_window_size_GRCh38_cnv_threshold_intersect_ONT_1000_window_size_cnv_threshold.bed | awk '{sum+=$3-$2} END {print sum}'

In [ ]:
### HG005_GRCh38_union_PacBio_HiFi_ONT_1000_window_size_combined_elliptical_outlier_threshold.bed

### What this does: Find another set of potential CNVs from computing a coverage threshold using an elliciptal outlier for PacBio HiFi and ONT CNV file steps in R

mosdepth_CCS_15kb_20kb_merged_1000_window_size_GRCh38 = read.delim("HG005_GRCh38_PacBio_HiFi_1000_window_size_MAPQ20.regions.bed", col.names = c("CHR","START","END","DEPTH"))

mosdepth_ONT_1000_window_size_GRCh38 = read.delim("HG005_GRCh38_ONT_filtered_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

chr_1_22 <- c("chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22")
mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22_GRCh38 <- mosdepth_CCS_15kb_20kb_merged_1000_window_size_GRCh38[which(mosdepth_CCS_15kb_20kb_merged_1000_window_size_GRCh38[,"CHR"] %in% chr_1_22),]

mosdepth_ONT_1000_window_size_chr_1_22_GRCh38 <- mosdepth_ONT_1000_window_size_GRCh38[which(mosdepth_ONT_1000_window_size_GRCh38[,"CHR"] %in% chr_1_22),]

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_GRCh38 <- data.frame(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22_GRCh38)
df_mosdepth_ONT_1000_window_size_GRCh38 <- data.frame(mosdepth_ONT_1000_window_size_chr_1_22_GRCh38)

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_GRCh38 <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_GRCh38

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_GRCh38[,5] <- mosdepth_ONT_1000_window_size_chr_1_22_GRCh38[,4]

colnames(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_GRCh38) <- c("CHR", "START", "END", "CCS_DEPTH", "ONT_DEPTH")

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values_GRCh38 <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_GRCh38
medianccsdepth = median(mosdepth_CCS_15kb_20kb_merged_1000_window_size_chr_1_22_GRCh38[,"DEPTH"])
medianontdepth = median(mosdepth_ONT_1000_window_size_chr_1_22_GRCh38[,"DEPTH"])
df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values_GRCh38[,6] <- sqrt(((df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_GRCh38[,4]/medianccsdepth)^2 + (df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_GRCh38[,5]/medianccsdepth)^2)/2)
threshold_ellipctial_outlier = unname(quantile(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values_GRCh38[,6])[4]+(1.5*IQR(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values_GRCh38[,6])[1]))

df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_outliers_GRCh38 <- df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values_GRCh38[which(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_elliptical_values_GRCh38[,6] > threshold_ellipctial_outlier),]

write.csv(df_mosdepth_CCS_15kb_20kb_merged_1000_window_size_ONT_1000_combined_outliers_GRCh38, file = "HG005_GRCh38_union_PacBio_HiFi_ONT_1000_window_size_combined_elliptical_outlier_threshold.bed", row.names = FALSE)

In [ ]:
## GRCh38 generate SV exclusion bed taking union of >=50bp SVs from dipcall, svanalyzer, and pbsv

HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns.vcf and HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns.vcf from Nancy Hansen on T2T globus


HG005.GRCh38.pbsv.vcf from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/ChineseTrio/analysis/PacBio_CCS_15kb_20kb_chemistry2_12072020/HG005/HG005.GRCh38.pbsv.vcf.gz

HG005_hifiasm.dip.vcf.vcf from https://drive.google.com/drive/folders/1m5RPh3rXBFNF3k_KnveVIaK6D5d96x_k

In [ ]:
python convert_vcf_to_bed_svanalyzer_gt_50bp.py --input HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns.vcf --output HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp.bed

python convert_vcf_to_bed_svanalyzer_gt_50bp.py --input HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns.vcf --output HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp.bed

python convert_vcf_to_bed_pbsv.py --input HG005.GRCh38.pbsv.vcf --output HG005.GRCh38.pbsv.bed

python convert_vcf_to_bed_dipcall.py --input HG005_hifiasm.dip.vcf --output HG005_GRCh38_hifiasm_gt50bp.bed



cat HG005_GRCh38_hifiasm_gt50bp.bed | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' > HG005_GRCh38_hifiasm_gt50bp_sorted.bed

cat HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp.bed | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' > HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted.bed

cat HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp.bed | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' > HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted.bed

cat HG005.GRCh38.pbsv_gt50bp.bed | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' > HG005.GRCh38.pbsv_gt50bp_sorted.bed


python remove_all_but_chr1_22_w_chr.py --input HG005_GRCh38_hifiasm_gt50bp_sorted.bed --output HG005_GRCh38_hifiasm_gt50bp_sorted_chr1_22.bed

python remove_all_but_chr1_22_w_chr.py --input HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted.bed --output HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted_chr1_22.bed

python remove_all_but_chr1_22_w_chr.py --input HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted.bed --output HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted_chr1_22.bed

python remove_all_but_chr1_22_w_chr.py --input HG005.GRCh38.pbsv_gt50bp_sorted.bed --output HG005.GRCh38.pbsv_gt50bp_sorted_chr1_22.bed


multiIntersectBed -i HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted_chr1_22.bed HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_refwidened_gt_50bp_sorted_chr1_22.bed HG005.GRCh38.pbsv_gt50bp_sorted_chr1_22.bed HG005_GRCh38_hifiasm_gt50bp_sorted_chr1_22.bed | mergeBed -i stdin | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' > HG005_GRCh38_union_SVs.bed

cat HG005_GRCh38_union_SVs.bed | awk '{sum+=$3-$2} END {print sum}'


cat GRCh38_AllTandemRepeatsandHomopolymers_slop5.bed | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' > GRCh38_AllTandemRepeatsandHomopolymers_slop5_sorted.bed

intersectBed -wa -a GRCh38_AllTandemRepeatsandHomopolymers_slop5_sorted.bed -b HG005_GRCh38_union_SVs.bed | multiIntersectBed -i stdin HG005_GRCh38_union_SVs.bed | awk '{FS=OFS="\t"} {print $1,$2-50,$3+50}' | mergeBed -i stdin -d 1000 > HG005_GRCh38_union_SVs_repeatexpanded_slop50_merge1000.bed

cat HG005_GRCh38_union_SVs_repeatexpanded_slop50_merge1000.bed | awk '{sum+=$3-$2} END {print sum}'

bedtools slop -i HG005_GRCh38_union_SVs_repeatexpanded_slop50_merge1000.bed -g human.hg38.genome -b 0.25 -pct | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' | mergeBed -i stdin > HG005_GRCh38_union_SVs_repeatexpanded_slop50_merge1000_expanded_150_percent.bed

cat HG005_GRCh38_union_SVs_repeatexpanded_slop50_merge1000_expanded_150_percent.bed | awk '{sum+=$3-$2} END {print sum}'


In [ ]:
### Alignment of assemblies to identify potential CNV

Nancy Hansen provided an both assembly coverage bedgraphs of maternal and paternal haplotypes in December 2019

In [ ]:
# HG2_SKor_TrioONTCanu
unionBedGraphs -i HG005.pat_T2T_hifiasm_l100c1000.ps.1delta.sort.renamedcontigs.bedgraph HG005.mat_T2T_hifiasm_l100c1000.ps.1delta.sort.renamedcontigs.bedgraph > HG005.T2T_hifiasm_l100c1000.ps.1delta.sort.renamedcontigs.tempunion.bedgraph

python find_union.py --input HG005.T2T_hifiasm_l100c1000.ps.1delta.sort.renamedcontigs.tempunion.bedgraph --output HG005.T2T_hifiasm_l100c1000.union.qdelta.sort.renamedcontigs.bedgraph

python remove_all_but_chr1_22_w_chr.py --input HG005.T2T_hifiasm_l100c1000.union.qdelta.sort.renamedcontigs.bedgraph  --output HG005.T2T_hifiasm_l100c1000.union.qdelta.sort.renamedcontigs.chr1_22.bedgraph

## Intersect and find > 10kb

cat HG005.T2T_hifiasm_l100c1000.union.qdelta.sort.renamedcontigs.chr1_22.bedgraph | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' | bedtools intersect -a stdin -b HG005.T2T_hifiasm_l100c1000.union.qdelta.sort.renamedcontigs.chr1_22.bedgraph | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' | mergeBed -i stdin -d 100  | awk '$3-$2>10000' > HG005.T2T_hifiasm_l100c1000.union_gt10kb_GRCh38.bed

In [ ]:
find_union.py

import argparse

parser = argparse.ArgumentParser(description="Subset bed file to >1 for column 4")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output', metavar="O", type=str, nargs="+", help="output file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_lines = f.readlines()

f_out = open(args.output[0], "w+")
number_entries_covered = 0
for line in f_lines:
    split_line = line.split("\t")
    if int(split_line[3].strip()) > 1 or int(split_line[4].strip()) > 1:
        number_entries_covered = number_entries_covered + int(split_line[3])
        f_out.write(line)
        f_out.flush()

f.close()
f_out.close()



In [ ]:
remove_all_but_chr1_22_w_chr.py 

import argparse

parser = argparse.ArgumentParser(description="Remove all but chr1-22 entries")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input file")
parser.add_argument('--output', metavar="O", type=str, nargs="+", help="output file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_lines = f.readlines()
chrs_to_keep = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22"]

f_out = open(args.output[0], "w+")

for line in f_lines:
    split_line = line.split("\t")
    if split_line[0] in chrs_to_keep:
        f_out.write(line)
        f_out.flush()

f.close()
f_out.close()

In [ ]:
### Inversions 

Nancy Hansen provided SVanalyzer calls and extracted inversions that were identified in HG005 in January 2020

In [ ]:
grep -e "^#" -e "INV" HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns.vcf > HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS.vcf

grep -e "^#" -e "INV" HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns.vcf > HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS.vcf

python convert_inversions_vcf_to_bed.py --input HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS.vcf --output HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened.bed

python convert_inversions_vcf_to_bed.py --input HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS.vcf --output HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened.bed

cat HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened.bed | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' > HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened_sorted.bed

cat HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened.bed | sed 's/^chr//' | sort -k1,1 -k2,2n | sed 's/^/chr/' > HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened_sorted.bed

multiIntersectBed -i HG005.pat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened_sorted.bed HG005.mat_T2T_hifiasm.l100c1000.ps.no_ns_INVERSIONS_refwidened_sorted.bed > HG005_inversions.bed


bedtools slop -i HG005_inversions.bed -g human.hg38.genome -b 0.25 -pct > HG005_inversions_slop150.bed

python remove_all_but_chr1_22_w_chr.py --input HG005_inversions_slop150.bed --output HG005_inversions_slop150_chr1_22.bed



In [ ]:
convert_inversions_vcf_to_bed.py

import argparse

parser = argparse.ArgumentParser(description="Convert inversions VCF to bed")
parser.add_argument('--input', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output', metavar="O", type=str, nargs="+", help="output bed file")
args = parser.parse_args()

f = open(args.input[0], "r") 
f_out = open(args.output[0], "w+")

lines = f.readlines()

for line in lines:
    if "#" in line:
        continue
    split_line = line.split("\t")
    info_field = split_line[7]
    info_field_split = info_field.split(";")
    chrom = "0"
    start = "0"
    end = "0"
    clusterIDs = ""
    for split in info_field_split:
        if "REFWIDENED" in split:
            refwidened_split = split.split("=")
            refwidened_split_on_colon = refwidened_split[1].split(":")
            chrom = refwidened_split_on_colon[0]
            refwidened_split_start_end = refwidened_split_on_colon[1].split("-")
            start = refwidened_split_start_end[0]
            end = refwidened_split_start_end[1]
        elif "ClusterIDs" in split:
            clusterIDs = split
    to_write_out = chrom + "\t" + start + "\t" + end + "\t" + clusterIDs + "\n"
    f_out.write(to_write_out)
    f_out.flush()

f.close()
f_out.close()

In [ ]:
### Expand GIAB SV v0.6 by 50%

In [ ]:
bedtools slop -i HG005_union_SVs.bed -g GRCh38_genome.dict -b 0.25 -pct > HG005_union_SVs_slop150.bed

convert_mosdepth_to_excessive_coverage.py 

import argparse

parser = argparse.ArgumentParser(description="Subset bed file to callable regions only")
parser.add_argument('--input_file', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output_file', metavar="O", type=str, nargs="+", help="output file")
parser.add_argument('--threshold', metavar="T", type=str, nargs="+", help="input threshold")
args = parser.parse_args()

f = open(args.input_file[0], "r") 
f_lines = f.readlines()

f_out = open(args.output_file[0], "w+")
threshold = float(args.threshold[0])

for line in f_lines:   
    if "DEPTH" in line: 
        continue
    line_split = line.split("\t")
    depth_field = float(line_split[3])
    if depth_field > threshold:
        f_out.write(line)
        f_out.flush()  

f.close()
f_out.close()